In [1]:
import numpy as np
import pandas as pd
import re

import warnings
warnings.filterwarnings("ignore")

the dataset contains THREE YEARS of recorded incidents ("alarms") of broken fridges with some specific infos:
* When the fridge breaks down this is reported in the computer system and the CMS company sends an ingeneer to fix it and they it asks the Supermarket to pay for the service (maybe its a good business in Colombia ;)
* the original file has 150MB
* 600.000 records from 2012-2015

The questions are:
* If the company has details (data, logs etc) on all the "alarms" in the year 2012 - 2015, it wants to predicts how many alarms in each store in each month will happen in 2016?
* why?
* which shops has largest smallest number of alarms (maybe normalize per size of the shop)
* what data would you suggest the manager to collect to make the dataser more complete
* report and ask questions


Lets look at the data
* Work Order Number ----------------------------------------- 12939001 (like a factura, work done was stored as a number)
* Site Number -------------------------------------------------- A0052 (each store that company maintains has a number)
* Site Type ---------------------------------------------- SUPERMARKET (type of store: Supermarket(big)/Convenience(small))
* Date Created -------------------------------------- 02/02/2012 09:45 (date and time that the alarm occured)
* Date Completed  ----------------------------------- 02/02/2012 21:10 (date and time that the alarm was silenced)
* Provider Name  ------------------------------- CMS Solutions Limited (name of the maintenance company)
* WO Current Status ---------------------------------------------- EOL (I dont remember)
* Asset Category --------------------------------------- REFRIGERATION (type of asset, they also fix aircond, light etc)
* Problem Type Name ---------- Fixture Critical Over Temperature Alarm (type of alarm)
* Requested By ---------------------------- Urgent Refrigeration Alarm (priority of the alarm)
* WO Priority ------------------------------------------- Standard SLA (for accountancy of the company)
* Manufacturer ---------------------------------------------- LINDE AG (manufacturer of the asset that made the alarm)
* Asset In-Service Date ----------------------------- 23/05/2012 05:00 (when the machine was first connected)
* Level 1 Organization Name ------------------------------- Region S01 (the company divied the country into regions)
* Site Zipcode ----------------------------------------------- CR0 4XT (postal code of the store - helps to locate it)
* Sales Sq. Ft. ---------------------------------------------- 48409.0 (the size of the stor in square feet)
* Asset Barcode ------------------------------------------ SSL00319496 (number of the assed (fridge), a label)
* Client Product Name -------------------------------------------- NaN (dont remember)
* Asset Type ---------------------------------------- REMOTE MULTIDECK (fridge can be open, closed, cabinet etc)
* Asset Status ---------------------------------------------- Inactive (is the asset active today -time of generating dataset)

In [2]:
data = pd.read_csv("data/CPT_PREDICTIVE_PPM.csv",encoding='windows-1252')
data.head()

,Work Order Number,Site Number,Site Type,Date Created,Date Completed,Provider Name,WO Current Status,Asset Category,Problem Type Name,Requested By,WO Priority,Manufacturer,Asset In-Service Date,Level 1 Organization Name,Site Zipcode,Sales Sq. Ft.,Asset Barcode,Client Product Name,Asset Type,Asset Status
0,12904684,A0892,SUPERMARKET,01/02/2012 03:05,01/02/2012 10:00,Space Engineering services LTD,EOL,REFRIGERATION,Fixture Critical Over Temperature Alarm,Urgent Refrigeration Alarm,Standard SLA,CARTER,01/06/2011 15:19,Region N10,DN31 1UF,42639.0,SSL00567391,Frozen Food (-18C or less),HALF GLASS DOOR & WELL,In Service
1,12904685,A0892,SUPERMARKET,01/02/2012 03:05,01/02/2012 09:59,Space Engineering services LTD,EOL,REFRIGERATION,Fixture Critical Over Temperature Alarm,Urgent Refrigeration Alarm,Standard SLA,CARTER,01/06/2011 15:19,Region N10,DN31 1UF,42639.0,SSL00567398,Frozen Food (-18C or less),FULL GLASS DOOR,In Service
2,12904853,A0892,SUPERMARKET,01/02/2012 05:10,01/02/2012 10:01,Space Engineering services LTD,EOL,REFRIGERATION,Fixture Critical Over Temperature Alarm,Urgent Refrigeration Alarm,Standard SLA,CARTER,01/06/2011 15:19,Region N10,DN31 1UF,42639.0,SSL00567399,Frozen Food (-18C or less),FULL GLASS DOOR,In Service
3,12927899,A0416,SUPERMARKET,01/02/2012 08:40,02/02/2012 13:03,CMS Solutions Limited,EOL,REFRIGERATION,Fixture Critical Over Temperature Alarm,Urgent Refrigeration Alarm,Standard SLA,LINDE,01/08/2002 05:00,Region S01,BR4 0PU,13423.0,SSL00225160,"Meat, Fish and Poultry (3C or less)",REMOTE MULTIDECK,In Service
4,12932231,A0003,SUPERMARKET,01/02/2012 13:33,08/02/2012 19:39,CMS Solutions Limited,EOL,REFRIGERATION,Fixture Critical Over Temperature Alarm,Urgent Refrigeration Alarm,Standard SLA,RADFORD RETAIL SYSTEMS,15/03/2009 04:00,Region S02,DA11 8JH,37870.0,SSL00583503,NaN,FULL GLASS DOOR,Scrapped


### Organizar columnas para llamar posteriormente

In [9]:
data.columns[0].split(" ")

['work', 'order', 'number']

In [18]:
col = []
for columns in data.columns:
    temp = columns.replace(" ","_")
    temp = columns.replace("-","_")
    temp = columns.replace(".","")
    temp = temp.lower()
    col.append(temp)
data.columns = col
data.head()

,work_order_number,site_number,site_type,date_created,date_completed,provider_name,wo_current_status,asset_category,problem_type_name,requested_by,wo_priority,manufacturer,asset_in_service_date,level_1_organization_name,site_zipcode,sales_sq_ft,asset_barcode,client_product_name,asset_type,asset_status
0,12904684,A0892,SUPERMARKET,2012-01-02 03:05:00,2012-01-02 10:00:00,Space Engineering services LTD,EOL,REFRIGERATION,Fixture Critical Over Temperature Alarm,Urgent Refrigeration Alarm,Standard SLA,CARTER,01/06/2011 15:19,Region N10,DN31 1UF,42639.0,SSL00567391,Frozen Food (-18C or less),HALF GLASS DOOR & WELL,In Service
1,12904685,A0892,SUPERMARKET,2012-01-02 03:05:00,2012-01-02 09:59:00,Space Engineering services LTD,EOL,REFRIGERATION,Fixture Critical Over Temperature Alarm,Urgent Refrigeration Alarm,Standard SLA,CARTER,01/06/2011 15:19,Region N10,DN31 1UF,42639.0,SSL00567398,Frozen Food (-18C or less),FULL GLASS DOOR,In Service
2,12904853,A0892,SUPERMARKET,2012-01-02 05:10:00,2012-01-02 10:01:00,Space Engineering services LTD,EOL,REFRIGERATION,Fixture Critical Over Temperature Alarm,Urgent Refrigeration Alarm,Standard SLA,CARTER,01/06/2011 15:19,Region N10,DN31 1UF,42639.0,SSL00567399,Frozen Food (-18C or less),FULL GLASS DOOR,In Service
3,12927899,A0416,SUPERMARKET,2012-01-02 08:40:00,2012-02-02 13:03:00,CMS Solutions Limited,EOL,REFRIGERATION,Fixture Critical Over Temperature Alarm,Urgent Refrigeration Alarm,Standard SLA,LINDE,01/08/2002 05:00,Region S01,BR4 0PU,13423.0,SSL00225160,"Meat, Fish and Poultry (3C or less)",REMOTE MULTIDECK,In Service
4,12932231,A0003,SUPERMARKET,2012-01-02 13:33:00,2012-08-02 19:39:00,CMS Solutions Limited,EOL,REFRIGERATION,Fixture Critical Over Temperature Alarm,Urgent Refrigeration Alarm,Standard SLA,RADFORD RETAIL SYSTEMS,15/03/2009 04:00,Region S02,DA11 8JH,37870.0,SSL00583503,NaN,FULL GLASS DOOR,Scrapped


In [19]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 609989 entries, 0 to 609988
Data columns (total 20 columns):
 #   Column                     Non-Null Count   Dtype         
---  ------                     --------------   -----         
 0   work_order_number          609989 non-null  int64         
 1   site_number                609989 non-null  object        
 2   site_type                  609989 non-null  object        
 3   date_created               609989 non-null  datetime64[ns]
 4   date_completed             577602 non-null  datetime64[ns]
 5   provider_name              609989 non-null  object        
 6   wo_current_status          609989 non-null  object        
 7   asset_category             609989 non-null  object        
 8   problem_type_name          609989 non-null  object        
 9   requested_by               527644 non-null  object        
 10  wo_priority                609989 non-null  object        
 11  manufacturer               609989 non-null  object  

Se observan 7 columnas incompletas, se revisan primero estas:

### **date_created | date_completed | asset_in_service_date**

In [20]:
data.date_completed[0] # formato: Fecha más horas y minutos	YYYY-MM-DDThh:mmTZD

Timestamp('2012-01-02 10:00:00')

In [21]:
print( "Numero de valores en 'date_completed' con nan:",data[data.date_completed.isnull()]["work_order_number"].count() ) # valores con NaN en date_completed
print( "Porcentaje de datos perdidos respecto al tamaño del Dataset: "+str(round((32387/609989)*100 ,2))+"%" )
print( "Formato de valores en la columna:",type(data.date_completed[0]) )

Numero de valores en 'date_completed' con nan: 32387
Porcentaje de datos perdidos respecto al tamaño del Dataset: 5.31%
Formato de valores en la columna: <class 'pandas._libs.tslibs.timestamps.Timestamp'>


In [22]:
%%time
data["date_created"] = pd.DatetimeIndex(data.date_created)
data["date_completed"] = pd.DatetimeIndex(data.date_completed)
data["asset_in_service_date"] = pd.DatetimeIndex(data.asset_in_service_date)

CPU times: total: 17 s
Wall time: 42.7 s


In [24]:
data[data.date_completed.isnull()].head(3)

,work_order_number,site_number,site_type,date_created,date_completed,provider_name,wo_current_status,asset_category,problem_type_name,requested_by,wo_priority,manufacturer,asset_in_service_date,level_1_organization_name,site_zipcode,sales_sq_ft,asset_barcode,client_product_name,asset_type,asset_status
13,12940592,A0003,SUPERMARKET,2012-02-02 12:02:00,NaT,CMS Solutions Limited,Cancelled,REFRIGERATION,Fixture Critical Over Temperature Alarm,Urgent Refrigeration Alarm,Standard SLA,RADFORD RETAIL SYSTEMS,2009-03-15 04:00:00,Region S02,DA11 8JH,37870.0,SSL00583504,NaN,FULL GLASS DOOR,Inactive
15,12940828,A0003,SUPERMARKET,2012-02-02 12:26:00,NaT,CMS Solutions Limited,Cancelled,REFRIGERATION,Fixture Critical Over Temperature Alarm,Urgent Refrigeration Alarm,Standard SLA,ND,2009-03-15 04:00:00,Region S02,DA11 8JH,37870.0,SSL00836662 -duplicate-1,NaN,FULL GLASS DOOR,Scrapped
25,12953604,A2221,SUPERMARKET,2012-03-02 20:15:00,NaT,Space Engineering services LTD,Cancelled,REFRIGERATION,Fixture Critical Over Temperature Alarm,Urgent Refrigeration Alarm,Standard SLA,CRIOSBANC,2009-06-17 05:00:00,Region C01,GL2 5SA,NaN,SSL00624924,NaN,FISH COUNTER,In Service


Considerando que hay fechas que el mismo dia se activo y se silencio la alarma, entonces completamos los resultados faltantes de date_completed usando el metodo ffill de pandas el cual llena valores faltantes entre cada gap de nan por un valor valido.

In [25]:
data.date_completed.fillna(axis=0,method='ffill',inplace=True)

In [26]:
data[(data.work_order_number == 12940592) | (data.work_order_number == 12940828)]

,work_order_number,site_number,site_type,date_created,date_completed,provider_name,wo_current_status,asset_category,problem_type_name,requested_by,wo_priority,manufacturer,asset_in_service_date,level_1_organization_name,site_zipcode,sales_sq_ft,asset_barcode,client_product_name,asset_type,asset_status
13,12940592,A0003,SUPERMARKET,2012-02-02 12:02:00,2012-02-02 13:38:00,CMS Solutions Limited,Cancelled,REFRIGERATION,Fixture Critical Over Temperature Alarm,Urgent Refrigeration Alarm,Standard SLA,RADFORD RETAIL SYSTEMS,2009-03-15 04:00:00,Region S02,DA11 8JH,37870.0,SSL00583504,NaN,FULL GLASS DOOR,Inactive
15,12940828,A0003,SUPERMARKET,2012-02-02 12:26:00,2012-02-02 14:01:00,CMS Solutions Limited,Cancelled,REFRIGERATION,Fixture Critical Over Temperature Alarm,Urgent Refrigeration Alarm,Standard SLA,ND,2009-03-15 04:00:00,Region S02,DA11 8JH,37870.0,SSL00836662 -duplicate-1,NaN,FULL GLASS DOOR,Scrapped


Observamos que para las ordenes "12940592" y "12940828" el valor que completa los NaT son validos en comparacion a datos que estan completos. Similarmente ocurre con la columna "asset_in_service_date"

In [27]:
data[data.asset_in_service_date.isnull()].head(3)

,work_order_number,site_number,site_type,date_created,date_completed,provider_name,wo_current_status,asset_category,problem_type_name,requested_by,wo_priority,manufacturer,asset_in_service_date,level_1_organization_name,site_zipcode,sales_sq_ft,asset_barcode,client_product_name,asset_type,asset_status
3964,13342929,A0852,SUPERMARKET,2012-09-03 12:15:00,2012-10-03 12:36:00,Space Engineering services LTD,EOL,REFRIGERATION,Fixture Critical Over Temperature Alarm,Urgent Refrigeration Alarm,Standard SLA,LINDE,NaT,Region N11,LE11 5AS,28039.0,SSL00633639,NaN,WELL,Scrapped
4070,13346368,A0893,SUPERMARKET,2012-09-03 18:55:00,2012-10-03 18:02:00,Montgomery Refrigeration Ltd,Invoice Processed,REFRIGERATION,Fixture Critical Over Temperature Alarm,Urgent Refrigeration Alarm,Standard SLA,RMS CONTROLS,NaT,Region N03,G74 4UN,38116.0,SSL00017276,NaN,COLDROOM CONTROL PANEL,In Service
4743,13359685,A0893,SUPERMARKET,2012-11-03 21:05:00,2012-11-04 14:27:00,Montgomery Refrigeration Ltd,Invoice Processed,REFRIGERATION,Fixture Critical Over Temperature Alarm,Urgent Refrigeration Alarm,Standard SLA,RMS CONTROLS,NaT,Region N03,G74 4UN,38116.0,SSL00017276,NaN,COLDROOM CONTROL PANEL,In Service


In [28]:
data.asset_in_service_date.fillna(axis=0,method='ffill',inplace=True)

In [29]:
data[(data.work_order_number == 13342929) | (data.work_order_number == 13346368) | (data.work_order_number == 13359685)]

,work_order_number,site_number,site_type,date_created,date_completed,provider_name,wo_current_status,asset_category,problem_type_name,requested_by,wo_priority,manufacturer,asset_in_service_date,level_1_organization_name,site_zipcode,sales_sq_ft,asset_barcode,client_product_name,asset_type,asset_status
3964,13342929,A0852,SUPERMARKET,2012-09-03 12:15:00,2012-10-03 12:36:00,Space Engineering services LTD,EOL,REFRIGERATION,Fixture Critical Over Temperature Alarm,Urgent Refrigeration Alarm,Standard SLA,LINDE,2011-02-14 05:00:00,Region N11,LE11 5AS,28039.0,SSL00633639,NaN,WELL,Scrapped
4070,13346368,A0893,SUPERMARKET,2012-09-03 18:55:00,2012-10-03 18:02:00,Montgomery Refrigeration Ltd,Invoice Processed,REFRIGERATION,Fixture Critical Over Temperature Alarm,Urgent Refrigeration Alarm,Standard SLA,RMS CONTROLS,2009-05-15 05:00:00,Region N03,G74 4UN,38116.0,SSL00017276,NaN,COLDROOM CONTROL PANEL,In Service
4743,13359685,A0893,SUPERMARKET,2012-11-03 21:05:00,2012-11-04 14:27:00,Montgomery Refrigeration Ltd,Invoice Processed,REFRIGERATION,Fixture Critical Over Temperature Alarm,Urgent Refrigeration Alarm,Standard SLA,RMS CONTROLS,2009-06-26 17:12:00,Region N03,G74 4UN,38116.0,SSL00017276,NaN,COLDROOM CONTROL PANEL,In Service


Efectivamente asset_in_service_date debe ser un resultado menor que date_created dado la descripcion del dataset, por lo que es una buena aproximacion este resultado para completar los datos faltantes.

In [30]:
print( "Numero de valores en 'date_created' con nan:",data[data.date_created.isnull()]["work_order_number"].count() )
print( "Numero de valores en 'date_completed' con nan:",data[data.date_completed.isnull()]["work_order_number"].count() )
print( "Numero de valores en 'asset_in_service_date' con nan:",data[data.asset_in_service_date.isnull()]["work_order_number"].count() )

Numero de valores en 'date_created' con nan: 0
Numero de valores en 'date_completed' con nan: 0
Numero de valores en 'asset_in_service_date' con nan: 0


### **requested_by**

In [31]:
print(data.requested_by.unique()[:5])
print(len(data.requested_by.unique()))

['Urgent Refrigeration Alarm'
 'URGENT Refrigeration Fixture Communication Alarm'
 'URGENT Refrigeration Fixture Monitoring Alarm'
 'Valve Open >50% for 3hrs sustained for 3 times within 24hrs (All Cabinets and Coldrooms) LIVE '
 'AirOn/Off Temp Diff >5°C for 8hr within 24hr period(ProvCab) LIVE ']
112


La columna requested_by es la prioridad de la alarma, el dataset contiene 112 tipos de prioridad por lo que resulta complicado asignarle valores a los NaN. Esto significa de igual forma que si se realiza un one_hot_encoding resultaria en 112 columnas dificiles de procesar por el algoritmo de machine learning por lo que se cambiara los nan por una categoria mas llamada "No priority"

In [32]:
data.requested_by.fillna("No priority",inplace=True)
print(len(data.requested_by.unique()))

112


In [33]:
print( "Numero de valores en 'requested_by' con nan:",data[data.requested_by.isnull()]["work_order_number"].count() )

Numero de valores en 'requested_by' con nan: 0


### **sales_sq_ft**

In [34]:
type(data.sales_sq_ft[0])

numpy.float64

In [35]:
data[data.sales_sq_ft.isnull()].head(3)

,work_order_number,site_number,site_type,date_created,date_completed,provider_name,wo_current_status,asset_category,problem_type_name,requested_by,wo_priority,manufacturer,asset_in_service_date,level_1_organization_name,site_zipcode,sales_sq_ft,asset_barcode,client_product_name,asset_type,asset_status
25,12953604,A2221,SUPERMARKET,2012-03-02 20:15:00,2012-04-02 09:00:00,Space Engineering services LTD,Cancelled,REFRIGERATION,Fixture Critical Over Temperature Alarm,Urgent Refrigeration Alarm,Standard SLA,CRIOSBANC,2009-06-17 05:00:00,Region C01,GL2 5SA,NaN,SSL00624924,NaN,FISH COUNTER,In Service
26,12954198,A2221,SUPERMARKET,2012-04-02 02:00:00,2012-07-02 21:21:00,Space Engineering services LTD,EOL,REFRIGERATION,Fixture Critical Over Temperature Alarm,Urgent Refrigeration Alarm,Standard SLA,CRIOSBANC,2009-06-17 05:00:00,Region C01,GL2 5SA,NaN,SSL00624924,NaN,FISH COUNTER,In Service
58,13002141,A2167,SUPERMARKET,2012-07-02 15:41:00,2012-08-02 00:06:00,CMS Solutions Limited,EOL,REFRIGERATION,Fixture Critical Over Temperature Alarm,Urgent Refrigeration Alarm,Standard SLA,CARTER RETAIL EQUIPMENT LTD,2011-01-01 05:00:00,Region S02,CT21 5BH,NaN,SSL00507794,"Provisions, Dairy and Prepared Produce (5C or ...",REMOTE MULTIDECK,In Service


In [36]:
data[(data.site_number == "A2221") & (data.sales_sq_ft.isnull())].head(3)

,work_order_number,site_number,site_type,date_created,date_completed,provider_name,wo_current_status,asset_category,problem_type_name,requested_by,wo_priority,manufacturer,asset_in_service_date,level_1_organization_name,site_zipcode,sales_sq_ft,asset_barcode,client_product_name,asset_type,asset_status
25,12953604,A2221,SUPERMARKET,2012-03-02 20:15:00,2012-04-02 09:00:00,Space Engineering services LTD,Cancelled,REFRIGERATION,Fixture Critical Over Temperature Alarm,Urgent Refrigeration Alarm,Standard SLA,CRIOSBANC,2009-06-17 05:00:00,Region C01,GL2 5SA,NaN,SSL00624924,NaN,FISH COUNTER,In Service
26,12954198,A2221,SUPERMARKET,2012-04-02 02:00:00,2012-07-02 21:21:00,Space Engineering services LTD,EOL,REFRIGERATION,Fixture Critical Over Temperature Alarm,Urgent Refrigeration Alarm,Standard SLA,CRIOSBANC,2009-06-17 05:00:00,Region C01,GL2 5SA,NaN,SSL00624924,NaN,FISH COUNTER,In Service
277,13031014,A2221,SUPERMARKET,2012-11-02 01:55:00,2012-11-02 18:49:00,Space Engineering services LTD,EOL,REFRIGERATION,Fixture Critical Over Temperature Alarm,Urgent Refrigeration Alarm,Standard SLA,CARTER RETAIL EQUIPMENT,2009-06-17 05:00:00,Region C01,GL2 5SA,NaN,SSL00615845,Frozen Food (-18C or less),WELL,In Service


Indentificando el site_number (numero que representa la tienda) y buscando si tiene alojado algun nombre no logra identificar coincidencias, quiere decir que todos los resultados para esa tienda no contienen la columna "sales_sq_ft". como es un numero que representa el tamaño de la tienda entonces no se pueden completar los NaN con 0. debe ser un tamaño adecuado por lo que una aproximacion para las faltantes seria la media de la columna entera.

In [37]:
data.sales_sq_ft.fillna(value=data.sales_sq_ft.mean(),inplace=True)

In [38]:
print( "Numero de valores en 'sales_sq_ft' con nan:",data[data.sales_sq_ft.isnull()]["work_order_number"].count() )

Numero de valores en 'sales_sq_ft' con nan: 0


### **client_product_name**       

In [39]:
data[~data.client_product_name.isnull()].head(3)

,work_order_number,site_number,site_type,date_created,date_completed,provider_name,wo_current_status,asset_category,problem_type_name,requested_by,wo_priority,manufacturer,asset_in_service_date,level_1_organization_name,site_zipcode,sales_sq_ft,asset_barcode,client_product_name,asset_type,asset_status
0,12904684,A0892,SUPERMARKET,2012-01-02 03:05:00,2012-01-02 10:00:00,Space Engineering services LTD,EOL,REFRIGERATION,Fixture Critical Over Temperature Alarm,Urgent Refrigeration Alarm,Standard SLA,CARTER,2011-01-06 15:19:00,Region N10,DN31 1UF,42639.0,SSL00567391,Frozen Food (-18C or less),HALF GLASS DOOR & WELL,In Service
1,12904685,A0892,SUPERMARKET,2012-01-02 03:05:00,2012-01-02 09:59:00,Space Engineering services LTD,EOL,REFRIGERATION,Fixture Critical Over Temperature Alarm,Urgent Refrigeration Alarm,Standard SLA,CARTER,2011-01-06 15:19:00,Region N10,DN31 1UF,42639.0,SSL00567398,Frozen Food (-18C or less),FULL GLASS DOOR,In Service
2,12904853,A0892,SUPERMARKET,2012-01-02 05:10:00,2012-01-02 10:01:00,Space Engineering services LTD,EOL,REFRIGERATION,Fixture Critical Over Temperature Alarm,Urgent Refrigeration Alarm,Standard SLA,CARTER,2011-01-06 15:19:00,Region N10,DN31 1UF,42639.0,SSL00567399,Frozen Food (-18C or less),FULL GLASS DOOR,In Service


In [40]:
print(data.client_product_name.unique())
print(len(data.client_product_name.unique()))

['Frozen Food (-18C or less)' 'Meat, Fish and Poultry (3C or less)' nan
 'Provisions, Dairy and Prepared Produce (5C or less)'
 'Soft Drinks/BWS (5C or less - alarms at 12C)' 'Produce (7C or less)'
 'Pharmacy (2 - 8C)']
7


El nombre de la columna no tiene relacion con los valores que muestra, parece ser que es mas que todo una columna que muestra el tipo de uso que se le da a la nevera junto con su rango de enfriamiento. contiene 7 categorias (se puede separar una columna con las categorias y otra con el rango de enfriamiento)

In [41]:
type_of_use = []
cooling_range = []
for i in range(len(data)):
    try:
        type_temp = data.client_product_name[i].split("(")[0]
        max_cool_temp = re.findall("(-\d+|\d+)",data.client_product_name[i].split("(")[1])
        if len(max_cool_temp) > 1: cool_temp = f"({max_cool_temp[0]}-{max_cool_temp[1]})C - less"
        else: cool_temp = f"{max_cool_temp[0]}C - less"
        #print(cool_temp)
        type_of_use.append(type_temp)
        cooling_range.append(cool_temp)
    except:
        type_of_use.append("No information")
        cooling_range.append("No information")

data.insert(0,"preserves",type_of_use)
data.insert(0,"cooling_range",cooling_range)

In [42]:
data.head()

,cooling_range,preserves,work_order_number,site_number,site_type,date_created,date_completed,provider_name,wo_current_status,asset_category,...,wo_priority,manufacturer,asset_in_service_date,level_1_organization_name,site_zipcode,sales_sq_ft,asset_barcode,client_product_name,asset_type,asset_status
0,-18C - less,Frozen Food,12904684,A0892,SUPERMARKET,2012-01-02 03:05:00,2012-01-02 10:00:00,Space Engineering services LTD,EOL,REFRIGERATION,...,Standard SLA,CARTER,2011-01-06 15:19:00,Region N10,DN31 1UF,42639.0,SSL00567391,Frozen Food (-18C or less),HALF GLASS DOOR & WELL,In Service
1,-18C - less,Frozen Food,12904685,A0892,SUPERMARKET,2012-01-02 03:05:00,2012-01-02 09:59:00,Space Engineering services LTD,EOL,REFRIGERATION,...,Standard SLA,CARTER,2011-01-06 15:19:00,Region N10,DN31 1UF,42639.0,SSL00567398,Frozen Food (-18C or less),FULL GLASS DOOR,In Service
2,-18C - less,Frozen Food,12904853,A0892,SUPERMARKET,2012-01-02 05:10:00,2012-01-02 10:01:00,Space Engineering services LTD,EOL,REFRIGERATION,...,Standard SLA,CARTER,2011-01-06 15:19:00,Region N10,DN31 1UF,42639.0,SSL00567399,Frozen Food (-18C or less),FULL GLASS DOOR,In Service
3,3C - less,"Meat, Fish and Poultry",12927899,A0416,SUPERMARKET,2012-01-02 08:40:00,2012-02-02 13:03:00,CMS Solutions Limited,EOL,REFRIGERATION,...,Standard SLA,LINDE,2002-01-08 05:00:00,Region S01,BR4 0PU,13423.0,SSL00225160,"Meat, Fish and Poultry (3C or less)",REMOTE MULTIDECK,In Service
4,No information,No information,12932231,A0003,SUPERMARKET,2012-01-02 13:33:00,2012-08-02 19:39:00,CMS Solutions Limited,EOL,REFRIGERATION,...,Standard SLA,RADFORD RETAIL SYSTEMS,2009-03-15 04:00:00,Region S02,DA11 8JH,37870.0,SSL00583503,NaN,FULL GLASS DOOR,Scrapped


In [43]:
print(data.preserves.unique())
print(len(data.preserves.unique()))

['Frozen Food ' 'Meat, Fish and Poultry ' 'No information'
 'Provisions, Dairy and Prepared Produce ' 'Soft Drinks/BWS ' 'Produce '
 'Pharmacy ']
7


In [44]:
print(data.cooling_range.unique())
print(len(data.cooling_range.unique()))

['-18C - less' '3C - less' 'No information' '5C - less' '(5-12)C - less'
 '7C - less' '(2-8)C - less']
7


Ahora podemos eliminar la columna "client_product_name"    

In [45]:
data.drop("client_product_name",axis=1,inplace=True)

### **asset_type | asset_status**    

In [46]:
data[data.asset_status.isnull()]["cooling_range"].value_counts()

No information    83984
Name: cooling_range, dtype: int64

In [47]:
data[data.asset_type.isnull()]["cooling_range"].value_counts()

No information    82345
Name: cooling_range, dtype: int64

In [48]:
print(data.asset_status.unique())
print(len(data.asset_status.unique()))

['In Service' 'Scrapped' 'Inactive' 'Not Verified' 'Storage - Operable'
 'Planned Refit Removal' 'Not Found' 'Not Applicable' 'Storage - Hold'
 'Storage - Needs Review' 'Sold' nan]
12


In [49]:
data[data.asset_type.isnull()]["asset_status"].value_counts()

Series([], Name: asset_status, dtype: int64)

In [50]:
data[data.asset_status.isnull()]["asset_type"].value_counts()

No Asset Type    1639
Name: asset_type, dtype: int64

Se observa que en "asset_status" todos los valores que eran de "client_product_name" son nulos, ademas esta columna es a su vez categorica con 12 elementos, se cambiara los NaN por "No information".

En cambio hay 86 elementos dentro de "asset_type" donde todos los valores que eran de "client_product_name" son nulos y tambien los de "asset_status". Se cambiara los NaN por "No information".

In [51]:
print( "Numero de valores en 'asset_status' con nan:",data[data.asset_status.isnull()]["work_order_number"].count() )
print( "Numero de valores en 'asset_type' con nan:",data[data.asset_type.isnull()]["work_order_number"].count() )

Numero de valores en 'asset_status' con nan: 83984
Numero de valores en 'asset_type' con nan: 82345


In [52]:
print( "Porcentaje de datos perdidos respecto al tamaño del Dataset: "+str(round((82345/609989)*100 ,2))+"%" )

Porcentaje de datos perdidos respecto al tamaño del Dataset: 13.5%


Aunque parece ser que elementos con "asset_type" nulos contiene mas nulos respecto a todo el dataset, parece adecuado eliminar esas filas pero se pierde aproximadamente el 13.5% de los datos totales que puede dañar el modelo.

In [53]:
data.asset_status.fillna(value="No information",inplace=True)
data.asset_type.fillna(value="No information",inplace=True)

In [54]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 609989 entries, 0 to 609988
Data columns (total 21 columns):
 #   Column                     Non-Null Count   Dtype         
---  ------                     --------------   -----         
 0   cooling_range              609989 non-null  object        
 1   preserves                  609989 non-null  object        
 2   work_order_number          609989 non-null  int64         
 3   site_number                609989 non-null  object        
 4   site_type                  609989 non-null  object        
 5   date_created               609989 non-null  datetime64[ns]
 6   date_completed             609989 non-null  datetime64[ns]
 7   provider_name              609989 non-null  object        
 8   wo_current_status          609989 non-null  object        
 9   asset_category             609989 non-null  object        
 10  problem_type_name          609989 non-null  object        
 11  requested_by               609989 non-null  object  

### Type de columnas

In [55]:
data.columns

Index(['cooling_range', 'preserves', 'work_order_number', 'site_number',
       'site_type', 'date_created', 'date_completed', 'provider_name',
       'wo_current_status', 'asset_category', 'problem_type_name',
       'requested_by', 'wo_priority', 'manufacturer', 'asset_in_service_date',
       'level_1_organization_name', 'site_zipcode', 'sales_sq_ft',
       'asset_barcode', 'asset_type', 'asset_status'],
      dtype='object')

In [56]:
for i in data.columns:
    print(data[i][0],"  ----  ",type(data[i][0]))

-18C - less   ----   <class 'str'>
Frozen Food    ----   <class 'str'>
12904684   ----   <class 'numpy.int64'>
A0892   ----   <class 'str'>
SUPERMARKET   ----   <class 'str'>
2012-01-02 03:05:00   ----   <class 'pandas._libs.tslibs.timestamps.Timestamp'>
2012-01-02 10:00:00   ----   <class 'pandas._libs.tslibs.timestamps.Timestamp'>
Space Engineering services LTD   ----   <class 'str'>
EOL   ----   <class 'str'>
REFRIGERATION   ----   <class 'str'>
Fixture Critical Over Temperature Alarm   ----   <class 'str'>
Urgent Refrigeration Alarm   ----   <class 'str'>
Standard SLA   ----   <class 'str'>
CARTER   ----   <class 'str'>
2011-01-06 15:19:00   ----   <class 'pandas._libs.tslibs.timestamps.Timestamp'>
Region N10   ----   <class 'str'>
DN31 1UF   ----   <class 'str'>
42639.0   ----   <class 'numpy.float64'>
SSL00567391   ----   <class 'str'>
HALF GLASS DOOR & WELL   ----   <class 'str'>
In Service   ----   <class 'str'>


No es necesario cambiar el type de mas columnas

In [57]:
data.to_csv("./data/datacl.csv",index=None)